# Zambrano Garofalo Junior Vidal


## Download Files


In [1]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

In [2]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

--2022-01-12 15:14:49--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: ‘datasource.zip’

datasource.zip      100%[===================>]   4.15K  --.-KB/s    in 0s      

2022-01-12 15:14:50 (468 MB/s) - ‘datasource.zip’ saved [4249/4249]



## Unzip Files


In [3]:
!unzip datasource.zip -d dealership_data

Archive:  datasource.zip
  inflating: dealership_data/used_car_prices1.csv  
  inflating: dealership_data/used_car_prices2.csv  
  inflating: dealership_data/used_car_prices3.csv  
  inflating: dealership_data/used_car_prices1.json  
  inflating: dealership_data/used_car_prices2.json  
  inflating: dealership_data/used_car_prices3.json  
  inflating: dealership_data/used_car_prices1.xml  
  inflating: dealership_data/used_car_prices2.xml  
  inflating: dealership_data/used_car_prices3.xml  


## Set Paths


In [4]:
tmpfile    = "dealership_temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "dealership_transformed_data.csv"   # file where transformed data is stored

## Extract


In [5]:
# Add the CSV extractdef extract_from_csv(file_to_process):
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe


In [6]:
# Add the JSON extract function below
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

In [7]:
# Add the XML extract function below, it is the same as the xml extract function above but the column names need to be renamed.
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        car_model = person.find("car_model").text
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True)
    return dataframe

In [8]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data

    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)

    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)

    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)

    return extracted_data

## Transform


In [9]:
# Add the transform function below
def transform(data):
        data['price'] = round(data.price, 2)
        return data

## Loading


In [10]:
# Add the load function below
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)   

## Logging


In [11]:
# Add the log function below
def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y' #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n') 

## Running ETL Process


In [19]:
log("ETL Job Started")

log("Extract phase Started")
extracted_data = extract()
print(extracted_data)
log("Extract phase Ended")


        car_model year_of_manufacture         price    fuel
0        alto 800                2017   4253.731343  Petrol
1            ciaz                2015  10223.880597  Diesel
2            ciaz                2015  11194.029851  Petrol
3          ertiga                2015   9104.477612  Petrol
4           dzire                2009   3358.208955  Petrol
..            ...                 ...           ...     ...
85     etios liva                2014   7089.552239  Diesel
86         innova                2017  29477.611940  Petrol
87       fortuner                2010  13805.970149  Diesel
88  corolla altis                2011   6492.537313  Petrol
89  corolla altis                2016  21268.656716  Petrol

[90 rows x 4 columns]


In [27]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
print(transformed_data)
log("Transform phase Ended")

log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

log("ETL Job Ended")

        car_model year_of_manufacture     price    fuel
0        alto 800                2017   4253.73  Petrol
1            ciaz                2015  10223.88  Diesel
2            ciaz                2015  11194.03  Petrol
3          ertiga                2015   9104.48  Petrol
4           dzire                2009   3358.21  Petrol
..            ...                 ...       ...     ...
85     etios liva                2014   7089.55  Diesel
86         innova                2017  29477.61  Petrol
87       fortuner                2010  13805.97  Diesel
88  corolla altis                2011   6492.54  Petrol
89  corolla altis                2016  21268.66  Petrol

[90 rows x 4 columns]
